<a href="https://colab.research.google.com/github/santoshapatil/Edx-All-Courses/blob/main/edx_all_Courses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Web Scrapping All Courses on edx.org**


### Loading Essential Packages

In [1]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
import re
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
#browser Options
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


In [ ]:
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(5)
driver.get(url)
page = driver.page_source
soup = BeautifulSoup(page, "html.parser")


### Loading Course Basic Details 
1.Based on my crawling of edx website, I found that courses are stacked into subjects like Chemistry,Design,etc. So I automated a code to extract each course based on it's subject.

2.There are some courses stacked into multiple subjects, I filtered them based on Course URL, since course URL is unique to each course

In [ ]:
subj={'Architecture',
'Art %26 Culture',
'Biology %26 Life Sciences',
'Business %26 Management',
'Chemistry',
'Communication',
'Computer Science',
'Data Analysis %26 Statistics',
'Design',
'Economics %26 Finance',
'Education %26 Teacher Training',
'Electronics',
'Energy %26 Earth Sciences',
'Engineering',
'Environmental Studies',
'Ethics',
'Food %26 Nutrition',
'Health %26 Safety',
'History',
'Humanities',
'Language',
'Law',
'Literature',
'Math',
'Medicine',
'Music',
'Philanthropy',
'Philosophy %26 Ethics',
'Physics',
'Science',
'Social Sciences'}

### Function to Get Basic Course details like Course Name, Course URL, Course Photo URL, University.

In [ ]:
def get_course(u,n_p):
  print(u)
  total_page=n_p
  driver = webdriver.Chrome(options=options)
  driver.implicitly_wait(1)
  driver.get(u)
  page = driver.page_source
  soup = BeautifulSoup(page, "html.parser")
  for i in range(1,total_page+1):
  #driver = webdriver.Chrome(options=options)
  #driver.implicitly_wait(5)
  
    go=i
    c_path = '//div[@class="discovery-card Verified and Audit col col-xl-3 mb-4 static-card-spacing"]/a[@class="discovery-card-link"]'
    c_p_pg=len(soup.find_all('a',class_='discovery-card-link'))
    print(c_p_pg)
    for j in range(c_p_pg):
    
    #Attaching Course Link
      c_url=soup.find_all('a',class_='discovery-card-link')[j]
      clink="https://www.edx.org" + c_url.get('href')
      if clink in course_link:
        dup.append(clink)
        continue
      else:
        course_link.append(clink)
    
    #Attaching course Name/Title
      cnn=soup.find_all('h3',class_="name-heading")[j]
      if cnn is None:
        continue
      else:
        cnn=cnn.get_text()
        course_name.append(cnn)

    #attaching University /name
      univ = soup.find_all('div',class_='provider')[j]
      partner=univ.find_all('span')[-2].get_text()
      university_name.append(partner)
    
    #attaching Course photo
    
      try:

        c_pic=soup.find_all('div',class_='d-card-hero')[j]
        cp=c_pic.find('img').get('src')
        course_photo_URL.append(cp)
      except:
        course_photo_URL.append("not-mentioned")
    if i!=total_page:
      driver.find_element_by_xpath('//button[@class="btn next page-link"]').click()
    
      WebDriverWait(driver, 7).until(EC.visibility_of_element_located((By.XPATH, c_path)))
    #wait for another 30 seconds for the page to load completely
      time.sleep(6)
      #driver.get(url)
      go=go+1
      check = '//button[@aria-label="Page ' + str(go) + ', Current Page"]'
      page = driver.page_source
      soup = BeautifulSoup(page, "html.parser")
    else:
      break
  print("done")

In [ ]:
course_name=[]

course_photo_URL=[]

university_name=[]

course_link=[]

dup=[]
for u in subj:
  subj_url="https://www.edx.org/search?subject="+u+"&tab=course"
  driver = webdriver.Chrome(options=options)
  driver.implicitly_wait(0.5)
  driver.get(subj_url)
  try:
    n_of_pgs = driver.find_element_by_xpath('(//button[@class="btn page-link"])')
    if n_of_pgs is None:
      get_course(subj_url,1)
    else:
      n_of_pgs = driver.find_element_by_xpath('(//button[@class="btn page-link"])[last()]').text
      n_of_pg = int(n_of_pgs)
      get_course(subj_url,n_of_pg)
  
  #Subject with less number of courses like for Subject: Philanthrophy with just three Courses
  except NoSuchElementException :
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    
    c_p_pg=len(soup.find_all('a',class_='discovery-card-link'))
    print(c_p_pg)
    for j in range(c_p_pg):

    
    #Attaching course Name/Title
      cnn=soup.find_all('h3',class_="name-heading")[j]
      if cnn is None:
        continue
      else:
        cnn=cnn.get_text()
        course_name.append(cnn)
    
    #Attaching Course Link
      c_url=soup.find_all('a',class_='discovery-card-link')[j]
      clink="https://www.edx.org" + c_url.get('href')
      course_link.append(clink)

    #attaching University /name
      univ = soup.find_all('div',class_='provider')[j]
      partner=univ.find_all('span')[-2].get_text()
      university_name.append(partner)

    
    
    #attaching Course photo
    
      try:

        c_pic=soup.find_all('div',class_='d-card-hero')[j]
        cp=c_pic.find('img').get('src')
        course_photo_URL.append(cp)
      except:
        course_photo_URL.append("not-mentioned")


  

In [ ]:
len(dup)

3024

In [ ]:
edx_courses_catalog=pd.DataFrame(({'course_name':course_name,
                              'course_link':course_link,
                              'university_name':university_name,
                              'course_photo_URL':course_photo_URL
                              }))



In [ ]:
#writing into a .csv file
edx_courses_catalog.to_csv('edx_courses_catalog.csv')

In [3]:
from google.colab import files
uploaded = files.upload()

import io
edx_courses_catalog = pd.read_csv(io.BytesIO(uploaded[r'edx_courses_catalog.csv']))
edx_courses_catalog.head()

Saving edx_courses_catalog.csv to edx_courses_catalog.csv


,Unnamed: 0,course_name,course_link,university_name,course_photo_URL
0,0,The ArchitecturalImagination…,https://www.edx.org/course/the-architectural-i...,HarvardX,https://prod-discovery.edx-cdn.org/media/cours...
1,1,The Path to Happiness:What Chinese PhilosophyT...,https://www.edx.org/course/the-path-to-happiness,HarvardX,https://prod-discovery.edx-cdn.org/media/cours...
2,2,Pyramids of Giza: AncientEgyptian Art andArcha...,https://www.edx.org/course/pyramids-of-giza-an...,HarvardX,https://prod-discovery.edx-cdn.org/media/cours...
3,3,"U.S. Public Policy: Social,Economic, and Forei...",https://www.edx.org/course/us-public-policy-so...,HarvardX,https://prod-discovery.edx-cdn.org/media/cours...
4,4,"Women Making History:Ten Objects, ManyStories…",https://www.edx.org/course/women-making-histor...,HarvardX,https://prod-discovery.edx-cdn.org/media/cours...


### Getting All Course Details

Visiting each course and the Below code automates and extracts details about each course

In [ ]:
course_length=[]
course_level=[]
course_subtitle=[]
course_price=[]
course_effort=[]
course_language=[]
course_price=[]
course_enrollments=[]
course_partner=[]
university_link=[]
course_photo=[]
course_url=[]
course_title=[]
for c,ind,nm,p in zip(edx_courses_catalog.course_link[0:770],edx_courses_catalog.index[0:770],edx_courses_catalog.course_name[0:770],edx_courses_catalog.course_photo_URL[0:770]):

  driver = webdriver.Chrome(options=options)
  driver.implicitly_wait(3)
  driver.get(c)
  page = driver.page_source
  soup = BeautifulSoup(page, "html.parser")
  print(ind)
  course_photo.append(p)
  course_title.append(nm)
  course_url.append(c) 
  
#Attaching Course Enrollments
  try:
    enrol=soup.find('div',id="js-number-enrolled-label")
    enroll=enrol.find_all('span')[1].get_text()
    course_enrollments.append(enroll)
  except:
    course_enrollments.append("not-mentioned")

  try:
    check=soup.find_all('li',class_="list-group-item d-flex row px-0")
    
    
    if check is None:
      course_length.append("not-mentioned")
      course_effort.append("not-mentioned")
      course_level.append("not-mentioned")
      course_language.append("not-mentioned")
      course_subtitle.append("not-mentioned")
      course_price.append("not-mentioned")
    else:
      lngg,subb,leen,efff,leev,pric=[],[],[],[],[],[]

      for che in check:
        
        leng,eff,lev,lng,sub,pri='','','','','',''
      #Attaching Course Length
        t=che.get_text()
        if "Length"in t:
          leng=t.lstrip('Length:')
          leen.append(leng)
          continue
      #Attaching Course Effort
        elif "Effort"in t:
          eff=t.lstrip('Effort:')
          efff.append(eff)
          continue
      #Attaching Course Level
        elif "Level"in t:
          lev=t.lstrip('Level:')         
          leev.append(lev)
          continue
      #Attaching Course Language
        elif "Language"in t:         
          leng=t.lstrip('Language:')
          lngg.append(leng)
          continue
      #Attaching Course Subtitles
        elif "Transcript"in t:
          sub=t.lstrip('Video Transcript:')
          subb.append(sub)
          continue        
      #Attaching Course Price
        elif "Price"in t:
          pri=t.lstrip('Price:FREE Add a Verified Certificate for')
          pric.append(pri)
          continue
        else:
          continue
      
      course_language.append(lngg)
      course_subtitle.append(subb)
      course_length.append(leen)
      course_price.append(pric)
      course_effort.append(efff)
      course_level.append(leev)    

      lngg=[]
      subb=[]
      leen=[]
      efff=[]
      leev=[]
      pric=[]

      if leen is None:
        course_length.append("not-mentioned")
      elif efff is None:
        course_effort.append("not-mentioned")
      elif leev is None:
        course_level.append("not-mentioned")
      elif lngg is None:
        course_language.append("not-mentioned")
      elif subb is None:
        course_subtitle.append("not-mentioned")
      elif pric is None:
        course_price.append("not-mentioned")

  except :
    
    course_length.append("not-mentioned")
    course_effort.append("not-mentioned")
    course_level.append("not-mentioned")
    course_language.append("not-mentioned")
    course_subtitle.append("not-mentioned")
    course_price.append("not-mentioned")



  #Get University Full name and Partner Link
  try:
    univ=soup.find_all('div',class_='partner')
    universit=[]
    uni_link=[]
    for u in univ:
      ltp=u.find('a',class_='link').get_text()
      universit.append(ltp)
      univ_link="https://www.edx.org"+u.find('a').get('href')
      uni_link.append(univ_link)
    
    course_partner.append(universit)
    university_link.append(uni_link)
  except:
    course_partner.append("not-mentioned")
    university_link.append("not-mentioned")

In [10]:
All_edx_courses_catalog=pd.DataFrame({'course_name':course_title,
                                        'course_url':course_url,
                                        'course_photo':course_photo,
                              'course_level':course_level,
                              'course_effort':course_effort,
                              'course_length':course_length,
                              'course_language':course_language,
                              'course_subtitle':course_subtitle,
                              'course_price':course_price,
                              'course_partner':course_partner,
                              'university_link':university_link,
                              'course_enrollments':course_enrollments})
All_edx_courses_catalog.to_csv('All_edx_courses_catalog.csv')

In [34]:
All_edx_courses_catalog.to_csv('All_edx_courses_catalog_del_tab.csv',sep='\t')

In [35]:
All_edx_courses_catalog

,course_name,course_url,course_photo,course_level,course_effort,course_length,course_language,course_subtitle,course_price,course_partner,university_link,course_enrollments
0,The ArchitecturalImagination…,https://www.edx.org/course/the-architectural-i...,https://prod-discovery.edx-cdn.org/media/cours...,['Introductory'],['3–5 hours per week'],['10 Weeks'],['English'],['English'],['$199 USD'],['Harvard University'],['https://www.edx.org/school/harvardx'],"415,051"
1,The Path to Happiness:What Chinese PhilosophyT...,https://www.edx.org/course/the-path-to-happiness,https://prod-discovery.edx-cdn.org/media/cours...,['Introductory'],['1–2 hours per week'],['13 Weeks'],['English'],['English'],['$99 USD'],['Harvard University'],['https://www.edx.org/school/harvardx'],"29,623"
2,Pyramids of Giza: AncientEgyptian Art andArcha...,https://www.edx.org/course/pyramids-of-giza-an...,https://prod-discovery.edx-cdn.org/media/cours...,['Introductory'],['2–4 hours per week'],['8 Weeks'],['English'],['English'],['$149 USD'],['Harvard University'],['https://www.edx.org/school/harvardx'],"127,414"
3,"U.S. Public Policy: Social,Economic, and Forei...",https://www.edx.org/course/us-public-policy-so...,https://prod-discovery.edx-cdn.org/media/cours...,['Introductory'],['2–4 hours per week'],['4 Weeks'],['English'],['English'],['$99 USD'],['Harvard University'],['https://www.edx.org/school/harvardx'],"46,472"
4,"Women Making History:Ten Objects, ManyStories…",https://www.edx.org/course/women-making-histor...,https://prod-discovery.edx-cdn.org/media/cours...,['Introductory'],['2–3 hours per week'],['8 Weeks'],['English'],['English'],['$139 USD'],['Harvard University'],['https://www.edx.org/school/harvardx'],"40,788"
...,...,...,...,...,...,...,...,...,...,...,...,...
3077,Introduction to DataAnalysis using Excel…,https://www.edx.org/course/introduction-to-dat...,https://prod-discovery.edx-cdn.org/media/cours...,['Introductory'],['2–4 hours per week'],['4 Weeks'],['English'],['English'],['$99 USD'],['Microsoft'],['https://www.edx.org/school/microsoft'],not-mentioned
3078,"Data, Analytics andLearning…",https://www.edx.org/course/data-analytics-and-...,https://prod-discovery.edx-cdn.org/media/cours...,['Intermediate'],[],['4 Weeks'],['English'],"['English', 'English']",[],['University of Texas at Arlington'],['https://www.edx.org/school/utarlingtonx'],not-mentioned
3079,Statistics: Unlocking theWorld of Data…,https://www.edx.org/course/statistics-unlockin...,https://prod-discovery.edx-cdn.org/media/cours...,['Introductory'],['4–6 hours per week'],['8 Weeks'],['English'],['English'],['$49 USD'],['The University of Edinburgh'],['https://www.edx.org/school/edinburghx'],not-mentioned
3080,Biostatistics for Big DataApplications…,https://www.edx.org/course/biostatistics-for-b...,https://prod-discovery.edx-cdn.org/media/cours...,['Introductory'],['2–3 hours per week'],['8 Weeks'],['English'],['English'],['$49 USD'],['The University of Texas Medical Branch'],['https://www.edx.org/school/utmbx'],not-mentioned
